* 请在环境变量中设置`DB_URI`指向数据库

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PyFin.api import *
from alphamind.api import *
from alphamind.strategy.strategy import Strategy, RunningSetting
from alphamind.portfolio.meanvariancebuilder import target_vol_builder

plt.style.use('ggplot')

## 1. Single Day Analysis
-----------------------

In [ ]:
ref_date = '2018-01-08'
engine = SqlEngine(os.environ['DB_URI'])
universe = Universe('zz800')

In [ ]:
codes = engine.fetch_codes(ref_date, universe)
total_data = engine.fetch_data(ref_date, 'EPS', codes, 906, industry='sw', risk_model='day')
all_styles = risk_styles + industry_styles + ['COUNTRY']

In [ ]:
risk_cov = total_data['risk_cov'][all_styles].values
factor = total_data['factor']
risk_exposure = factor[all_styles].values
special_risk = factor['srisk'].values

In [ ]:
sec_cov = risk_exposure @ risk_cov @ risk_exposure.T / 10000 + np.diag(special_risk ** 2) / 10000
sec_cov_df = pd.DataFrame(sec_cov, index=codes, columns=codes)
sec_cov_df.iloc[:5, :5]

### Portfolio Construction

* using `EPS` factor as alpha factor;
* short selling is forbiden;
* target of volatility for the activate weight is setting at 2.5% annually level.

In [ ]:
er = factor['EPS'].values
bm = factor['weight'].values
lbound = np.zeros(len(er))
ubound = bm + 0.01
cons_mat = np.ones((len(er), 1))
risk_targets = (bm.sum(), bm.sum())
target_vol = 0.025

status, p_er, p_weight = \
    target_vol_builder(er, sec_cov, bm, lbound, ubound, cons_mat, risk_targets, target_vol, target_vol)

In [ ]:
# check the result
print(f"total weight is {p_weight.sum(): .4f}")
print(f"portfolio activate weight forecasting vol is {np.sqrt((p_weight - bm) @ sec_cov @ (p_weight - bm)):.4f}")
print(f"portfolio er: {p_weight @ er:.4f} comparing with benchmark er: {bm @ er:.4f}")

## 2. Porfolio Construction: 2016 ~ 2018
-------------------------------

In [ ]:
"""
Back test parameter settings
"""

start_date = '2016-01-01'
end_date = '2018-02-08'

freq = '10b'
neutralized_risk = industry_styles
industry_name = 'sw_adj'
industry_level = 1
risk_model = 'short'
batch = 0
horizon = map_freq(freq)
universe = Universe('zz800')
data_source = os.environ['DB_URI']
benchmark_code = 906
target_vol = 0.05
weights_bandwidth = 0.02

In [ ]:
"""
Factor Model
"""

alpha_factors = {'f01': CSRank(LAST('EPS'))}
weights = dict(f01=1.)
alpha_model = ConstLinearModel(features=alpha_factors, weights=weights)

data_meta = DataMeta(freq=freq,
                         universe=universe,
                         batch=batch,
                         neutralized_risk=neutralized_risk,
                         risk_model='short',
                         pre_process=[winsorize_normal, standardize],
                         post_process=[standardize],
                         warm_start=0,
                         data_source=data_source)

In [ ]:
"""
Constraintes settings
"""

constraint_risk = ['SIZE', 'SIZENL', 'BETA']
total_risk_names = constraint_risk + ['benchmark', 'total']

b_type = []
l_val = []
u_val = []

previous_pos = pd.DataFrame()
rets = []
turn_overs = []
leverags = []

for name in total_risk_names:
    if name == 'benchmark':
        b_type.append(BoundaryType.RELATIVE)
        l_val.append(0.8)
        u_val.append(1.0)
    else:
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(0.0)
        u_val.append(0.0)

bounds = create_box_bounds(total_risk_names, b_type, l_val, u_val)

In [ ]:
"""
Running Settings
"""
running_setting = RunningSetting(universe,
                                 start_date,
                                 end_date,
                                 freq,
                                 benchmark=benchmark_code,
                                 weights_bandwidth=weights_bandwidth,
                                 rebalance_method='tv',
                                 bounds=bounds,
                                 target_vol=target_vol)

In [ ]:
"""
Strategy run
"""
strategy = Strategy(alpha_model, data_meta, running_setting)
ret_df, positions = strategy.run()

In [ ]:
ret_df[['excess_return', 'turn_over']].cumsum().plot(figsize=(14, 7),
                                                     title='Fixed freq rebalanced with target vol \
                                                     at {2}: {0} with benchmark {1}'.format(freq, benchmark_code, target_vol),
                                                     secondary_y='turn_over')